In [ ]:
%load_ext autoreload
%autoreload 2
#%pylab inline

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
import tensorflow as tf
import numpy as np
import itertools
import keras
import gym

from tqdm import tqdm
from wrappers.envs import AtariStackFrames
from utils.ae import AEConvKeras

In [ ]:
model = keras.models.load_model('models/model_20170209-171440/model_epoch99.h5')

In [ ]:
env = AtariStackFrames(gym.make('MsPacman-v0'), rgb=True)

In [ ]:
def collect_data(env=None, n_episodes=10, max_steps=5000):
    states = []
    for i in tqdm(range(n_episodes)):
        state = env.reset()
        states.append(state)
        terminal = False
        step = 0
        while not terminal and step < max_steps:
            action = env.action_space.sample()
            state, reward, terminal, _ = env.step(action)
            states.append(state)
            step += 1
    return np.array(states)

def run_episode(env=None, agent=None, max_steps=5000):
    state = env.reset()
    total_reward = 0.
    terminal = False
    step = 0
    while not terminal and step < max_steps:
        action = agent.predict(np.expand_dims(state, 0)).argmax(axis=1)
        #action = env.action_space.sample()
        state, reward, terminal, _ = env.step(action)
        total_reward += reward
        step += 1
    return total_reward

In [ ]:
res = []
for i in xrange(20):
    r = run_episode(env, model)
    print r
    res.append(r)
print 'mean: {:.3f}'.format(np.array(res).mean())

## temp

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [18]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model

In [4]:
inp = Input(shape=(1,))
out = Dense(32, activation='relu')(inp)
out = Dense(1)(out)

In [24]:
with tf.device('/job:ps/task:0'):
    print m.weights[0].device

In [28]:
print model.input
print model.output

Tensor("input_1:0", shape=(?, 1), dtype=float32)
Tensor("add_1:0", shape=(?, 1), dtype=float32)


In [5]:
model = Model(input=inp, output=out)

In [6]:
model.save('temp.h5')

In [26]:
with tf.device('/job:ps/task:0'):
    m = keras.models.load_model('temp.h5')

InvalidArgumentError: Cannot assign a device to node 'dense_2_b_3': Could not satisfy explicit device specification '/job:ps/task:0' because no devices matching that specification are registered in this process; available devices: /job:localhost/replica:0/task:0/cpu:0
Colocation Debug Info:
Colocation group had the following types and devices: 
Assign: CPU 
Identity: CPU 
VariableV2: CPU 
	 [[Node: dense_2_b_3 = VariableV2[container="", dtype=DT_FLOAT, shape=[1], shared_name="", _device="/job:ps/task:0"]()]]

Caused by op u'dense_2_b_3', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 589, in launch_instance
    app.star

In [ ]:
with tf.variable_scope('policy11'):
    m = keras.models.load_model('temp.h5')
    for var in m.weights:
        print var.op.name

In [ ]:
m.summary()

In [ ]:
m.layers[-2].output

In [ ]:
m.input

In [ ]:
with tf.variable_scope('policy'):
    out = Dense(128)(m.layers[-2].output)
    new = Model(input=m.layers[0].input, output=out)

In [ ]:
new.summary()

In [ ]:
for var in new.weights:
    print var.op.name

In [ ]:
m.layers[0].input